# 1. Load VGG16 Model

In [ ]:
import tensorflow as tf
import os

In [ ]:
#学習済みモデルのVGG16をインポート
from keras.applications.vgg16 import VGG16

In [ ]:
#weights="imagenet"で、VGG16モデルの重みをImageNetデータセットから事前に学習されたものに設定する
#全結合層以外のVGG16のlayerを用いる。

base_model = VGG16(weights="imagenet", include_top=False, input_shape=(256, 256, 3))

#base_modelの重みを更新不可に設定する
base_model.trainable = False 

#ディープラーニングモデルを出力する。
print("model summary ")

base_model.summary()

#重みが本当に更新不可か確認する
print("Is layer Not trainable weights?")
for layer in base_model.layers:
    print(layer, layer.trainable)

# 2. Fine Chuning Model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping#学習に変化がなければ、学習を止めるモジュール

In [ ]:
#sequential neural network modelを作成。model.addでlayerを追加する。

#空のシーケンシャルモデルを作成
#シーケンシャルモデル:ニューラルネットワークモデルを構築するための層の線形スタック
model = Sequential()

#VGG16のlayerから先にaddする
model.add(base_model)

#追加の畳み込み層の追加
model.add(Conv2D(64, kernel_size=3, padding="same", activation="relu"))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

#全結合層の追加
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu")) 
model.add(Dense(64, activation="relu")) 
model.add(Dropout(0.5))

#0,1出力が欲しいので、sigmoid間数を用いる
model.add(Dense(1, activation='sigmoid'))

In [ ]:
#vgg16のbase_modelの重みが更新不可能であり、
#追加したlayerの重みが更新可能か確認するコード
for layer in model.layers:
    print(layer, layer.trainable )

print(len(model.trainable_weights))

モデルのコンパイルを行う。

In [ ]:
#Adamオプティマイザーを使用
#tf.losses.BinaryCrossentropy()を指定して損失関数として二値交差エントロピーを使用する。
#評価関数としてaccuracyを使用する.

model.compile(
    optimizer= 'adam', 
    loss=tf.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
#モデルの構造を表示。
model.summary()

# 3. Remove dodgy images

In [ ]:
import cv2
import imghdr

In [1]:
#ディレクトリ名を指定
data_dir = 'dataset1' 

In [ ]:
#受けつける拡張子の指定
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
#data_dirの中身のディレクトリを出力する。
#毒あり毒なしそれぞれのディレクトリの名前が出力されれば成功、
for image_class in os.listdir(data_dir): 
    print(image_class)

In [ ]:
#data_dirの中に.DS_Storeがあった場合取り除くコード。
import os 

for image_class in os.listdir(data_dir):
    if image_class == '.DS_Store':
        file_path = os.path.join(data_dir, image_class)
        os.remove(file_path)
    else:
        print(image_class)

In [ ]:
#.DS_Storeが消えているのを確認
for image_class in os.listdir(data_dir): 
    print(image_class)

In [ ]:
#対象としていない拡張子と壊れている画像ファイルをremoveする。
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
             os.remove(image_path)

0と1がどのクラスに対応しているのかを出力する。

In [ ]:
class_mapping = {}  # クラス名とラベルの対応付けを格納する辞書

for label, image_class in enumerate(os.listdir(data_dir)):
    if image_class == '.DS_Store':
        file_path = os.path.join(data_dir, image_class)
        os.remove(file_path)
    else:
        class_name = image_class.split('_')[0]
        print(f"{class_name}: {label}")
        class_mapping[label] = class_name

# class_mapping辞書を修正して、ラベルを0と1に変更する。
class_mapping = {label: label - 1 for label in class_mapping}

# 4. Load Data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#指定されたディレクトリから画像データセットを作成
data= tf.keras.utils.image_dataset_from_directory('dataset1')

In [ ]:
#データセットの要素をnumpy配列として返すiteratorを作成。
data_iterator = data.as_numpy_iterator()

In [ ]:
#イテレータから次の要素を取得する
batch = data_iterator.next()

#batch0:画像データ
#batch1: 0 or1（毒あり or 毒なしラベル）

In [ ]:
#バッチの上から４つ選び、プロットする。
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# 5. Scale Data

In [ ]:
#x:image
#y:target
#scale data to 0 through 1

data = data.map(lambda x,y: (x/255, y))

In [ ]:
#データセットの要素をnumpy配列として返すiteratorを作成。
#イテレータから次のバッチを取得する

data.as_numpy_iterator().next()

# 6. Split Data

In [ ]:
#バッチ数をプリント
len(data)

In [ ]:
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.2)
test_size = len(data) - train_size - val_size

In [ ]:
以下の3変数の合計がlen(data)分になるようにする。

In [ ]:
len(train_size )

In [ ]:
len(val_size)

In [ ]:
len(test_size)

In [ ]:
#take():how much we take from dataset
#skip():usually,skip take() area 

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# 7. Train

In [ ]:
#ログファイルを保存するディレクトリを指定
logdir='logs'

In [ ]:
#fit間数にわたすため、コールバックを作成
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
#学習に変化がない場合、中断させる。今回の場合10エポック変化がない場合、ストップ。
early_stop = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
#epoc数20でモデルを学習させる。

hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback, early_stop])

# 8. Plot Performance

#lossとval_lossをプロット

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

#accuracyとval_accuracyをプロット

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 9. Evaluate by test

学習済みモデルの評価を行う

In [ ]:
from keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
#精度、再現率、および二値分類精度を計算し、
#真のラベルyと予測されたラベルyhatを比べる

for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
#それぞれの出力値は1に近いほどよい
print(pre.result(), re.result(), acc.result())

# 10. Save the Model

学習モデルの保存を行う。

In [ ]:
from keras.models import load_model

In [ ]:
#modelsフォルダに学習済みモデルを保存
model.save(os.path.join('models','snake_poisonClassifierFC1.h5'))

# 11.load the model

以下は正しくloadして動作するかをチェックするコード。

In [ ]:
#保存したモデルをloadする
new_model = load_model('models/snake_poisonClassifierFC1.h5')

In [ ]:
#精度、再現率、および二値分類精度を計算し、
#真のラベルyと予測されたラベルyhatを比べる
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
#それぞれの出力値は1に近いほどよい
print(pre.result(), re.result(), acc.result())